In [1]:
from basico import *
from cobra import Model
from cobra.io import read_sbml_model
import pandas as pd
import numpy as np
import logging 
from tqdm import tqdm
from scipy.integrate import solve_ivp
import cobra.util.solver


logging.getLogger('cobra').setLevel(logging.ERROR)
model_fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'

def generate_timecourse(model_fp=model_fp, start=0, stop=10, steps=100):
    model = load_model(model_fp)
    outputs = get_species(model=model).index.tolist()
    t = np.linspace(0, 10, 1000)
    tc = run_time_course_with_output(outputs, t[-1], values=t, model=model)
    return tc 


def generate_reaction_mappings(cobra_model, output_names):
    mappings = []
    for reaction in cobra_model.reactions:
        for name in output_names:
            rxn = [r.lower() for r in reaction.name.split(" ")]
            obs_name = name.split(" ")[0].lower()
            obs_type = name.split(" ")[-1]
            if obs_name in rxn:
                mapping = {}
                if "transcription" in rxn and obs_type == "mRNA":
                    mapping = {name: reaction.name}
                elif "translation" in rxn and obs_type == "protein":
                    mapping = {name: reaction.name}
                elif "degradation" in rxn:
                    if "transcripts" in rxn and obs_type == "mRNA":
                        mapping = {name: reaction.name}
                    elif "transcripts" not in rxn and obs_type == "protein":
                        mapping = {name: reaction.name}
                if mapping:
                    mappings.append(mapping)
    return mappings


def apply_ode_results_to_fba(time_point, ode_results, cobra_model, mappings):
    """
    Apply ODE results to COBRApy model reactions based on mappings.
    Args:
        time_point: Current time step in the ODE simulation
        ode_results: Time-series DataFrame with concentration data
        cobra_model: COBRApy model
        mappings: List of mappings between ODE outputs and COBRA reactions
    """
    for mapping in mappings:
        for ode_species, cobra_reaction in mapping.items():
            # Extract the concentration at the current time point
            concentration = ode_results.loc[time_point, ode_species]
            
            # Apply this concentration to the relevant reaction in COBRA model
            # For example, if it is a degradation reaction, you can set a lower bound
            reaction = cobra_model.reactions.get_by_id(cobra_reaction)
            
            # Modify reaction bounds based on concentration
            if 'degradation' in cobra_reaction:
                reaction.lower_bound = -concentration  # Assuming a negative flux for degradation
            else:
                # Example: You could modify the upper bound or other parameters
                reaction.upper_bound = concentration
                

result = generate_timecourse()
output_names = result.columns.tolist()
time_points = result.index.tolist()
cobra_model = read_sbml_model(model_fp)
reaction_mappings = generate_reaction_mappings(cobra_model, output_names)

ModuleNotFoundError: No module named 'tqdm'